## 벡터 클래스의 부활

In [29]:
from array import array
import math

class Vector2d:
    __match_args__ = ('x', 'y')

    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [2]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y)

3.0 4.0


In [3]:
x, y = v1
x, y

(3.0, 4.0)

In [4]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [5]:
abs(v1)

5.0

In [6]:
bool(v1)

True

## 9.3 대안 생성자

In [7]:
@classmethod # 클래스메서드는 다음과 같은 데커레이터가 붙는다. 
def frombytes(cls, octets): # self 매개변수 없이 클래스 자신이 cls 매개변수로 전달된다. 
    typecode = chr(octets[0])
    # memoryview를 생성하고 typecode를 이용해서 형을 변환한다. 
    memv = memoryview(octets[1:]).cast(typecode)
    # memoryview를 언패킹해서 생성자에 필요한 인수로 전달한다. 
    return cls(*memv)

## 9.4 @classmethod 와 @staticmethod

In [8]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args # 모든 위치 인수를 보여준다. 

    @staticmethod
    def statmeth(*args):
        return args # 모든 위치 인수를 보여준다. 

In [10]:
# 호출 방법과 무관하게 Demo 클래스를 첫 번째 인수로 받는다. 
Demo.klassmeth()

(__main__.Demo,)

In [11]:
Demo.klassmeth("spam")

(__main__.Demo, 'spam')

In [12]:
# 평범한 함수처럼 동작한다. 
Demo.statmeth()

()

In [13]:
Demo.statmeth('spam')

('spam',)

## 9.5 포맷된 출력

In [14]:
br1 = 1/2.43
br1

0.4115226337448559

In [15]:
format(br1, '0.4f')

'0.4115'

In [16]:
"1 BRF = {rate:0.2f} USD".format(rate=br1)

'1 BRF = 0.41 USD'

In [17]:
from datetime import datetime
now = datetime.now()
format(now, '%H:%M:%S')

'23:16:58'

In [18]:
"It's now {:%I:%M %p}".format(now)

"It's now 11:16 PM"

In [19]:
v1 = Vector2d(3, 4)
format(v1)

'(3.0, 4.0)'

In [20]:
format(v1, '.3f')

TypeError: unsupported format string passed to Vector2d.__format__

In [21]:
def __format__(self, fmt_spec=''):
    # 'p'로 끝나면 극좌표를 사용한다. 
    if fmt_spec.endswith('p'):
        fmt_spec = fmt_spec[:-1]
        coords = (abs(self), self.angle())
        outer_fmt = '<{}, {}>'
    else:
        # 직교좌표를 만든다.
        coords = self
        outer_fmt = '({}, {})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(*components)

In [27]:
format(Vector2d(1, 1), 'p')

'<1.4142135623730951, 0.7853981633974483>'

In [28]:
format(Vector2d(1, 1), '0.5fp')

'<1.41421, 0.78540>'

## 9.6 해시 가능한 Vector2d

In [30]:
v1.x, v1.y

(3.0, 4.0)

In [32]:
v1.x = 7

In [33]:
v1.x, v1.y

(7, 4.0)

In [34]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
hash(v1), hash(v2)

(7, 384307168202284039)

In [36]:
set([v1, v2])

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

## 9.7 파이썬에서의 비공개 속성과 보호된 속성

In [37]:
v1 = Vector2d(3,4)
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [38]:
v1._Vector2d__x

3.0

## 9.9 클래스 속성 오버라이드

In [39]:
v1 = Vector2d(1.1, 2.2)
dumpd = bytes(v1)
dumpd

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [40]:
len(dumpd)

17

In [41]:
v1.typecode = 'f'
dumpf = bytes(v1)
dumpf

b'f\xcd\xcc\x8c?\xcd\xcc\x0c@'

In [42]:
len(dumpf)

9

In [43]:
Vector2d.typecode

'd'